# **Distracted Driver Detection**

*Problem statement*: Given the dataset consisting of driver images in car and corresponding labels for 10 nos. categories (e.g. safe driving, texting, talking etc.), your task is to build a classification model to predict the category for that image.

I have used a pretrained model and tried to implement my code with the help of EfficientNets.

**Importing Required Libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.applications import EfficientNetB3
from keras.callbacks import ModelCheckpoint, EarlyStopping

* ImageDataGenerator class is used to ease out our working with the images.
* EfficientNet has been imported to increase our accuracy.
* EarlyStopping makes sure that we don't unnecessarily train our model further.
* Few filters are added from Keras too.

**Making Directories**

In [ ]:
base_dir = '../input/state-farm-distracted-driver-detection'
train_dir = os.path.join(base_dir, 'imgs/train/')
test_dir = os.path.join(base_dir, 'imgs/test/')
data = pd.read_csv(os.path.join(base_dir, 'driver_imgs_list.csv'))

**Reviewing Dataset**

In [ ]:
data.head()
class_count = data.classname.value_counts()
fig = class_count.plot(kind='bar')

**Working With ImageDataGenerator Class**

In [ ]:
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    validation_split=0.2)

test_gen = ImageDataGenerator()

train_data= train_gen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    seed=42,
    subset='validation'
)

**Making The Model**

Using EfficientNets to make our model

In [ ]:
def define_model(num_classes):
    inputs = Input(shape=(224,224,3))
    base_model = EfficientNetB3(include_top=False, weights='imagenet')(inputs)
    x = GlobalAveragePooling2D()(base_model)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
                  loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

model = define_model(10)
model.summary()

**Training Our Model**

In [ ]:
checkpoint_callback = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
es = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(train_data, epochs=20, validation_data=val_data, callbacks=[es, checkpoint_callback])

**Evaluating Our Model**

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['train', 'val'])

**Predictions**

In [ ]:
test_dir = os.path.join(base_dir, 'imgs')

test_data = test_gen.flow_from_directory(
    test_dir,
    shuffle=False,
    target_size=IMAGE_SIZE,
    classes=['test'],
    batch_size=BATCH_SIZE
)

In [ ]:
preds = model.predict(test_data)

In [ ]:
test_imgs = os.path.join(base_dir, "imgs/test")

test_ids = sorted(os.listdir(test_imgs))
pred_df = pd.DataFrame(columns = ['img','c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
for i in range(len(preds)):
    pred_df.loc[i, 'img'] = test_ids[i]
    pred_df.loc[i, 'c0':'c9'] = preds[i]
    
pred_df.to_csv('submission.csv', index=False)

**Completed**